# Exploration of London's Boroughs

In [42]:
#imports
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Notes to Marker
Hi, this is a code segment that is very similar to the lab. I'm just running a JSON to get the boroughs of London out, and exploring what I have in the JSON that will be of use in the final product.

In [43]:
with open('hood.json',  errors='ignore') as json_data:
    london_data = json.load(json_data)

In [44]:
london_data[0]

{'area': 'Barking and Dagenham',
 'latitude': 51.54527673,
 'longitude': 0.13352768,
 'population': 209000,
 'bars-and-pubs': 20,
 'rent': 943,
 'crime': 14638,
 'crime-per-pop': 70.03827751,
 'schools': 64,
 'independent-schools': 4,
 'tweets': [{'created_at': '05/04/2018 15:35',
   'screen_name': 'mrchrisnewton',
   'text': "If the Conservative Party actually treated their staff with a degree of gratitude after elections, CCHQ might've had someone who could've helped them in Barking &amp; Dagenham. ï£¿Ã¼Â§Ã®https://t.co/FYzOFNHX9v"},
  {'created_at': '05/04/2018 17:36',
   'screen_name': 'wallaceme',
   'text': 'Worth noting Liverpool Conservatives are hotly disputing @GuidoFawkesâ€šÃ„Ã´ story - but @ConHome has confirmed these problems in Manchester and Barking&amp;Dagenham. https://t.co/YZmNqGG7OG'},
  {'created_at': '08/04/2018 06:41',
   'screen_name': 'Andrewwood17',
   'text': "We could do with something like this on the IoD but it should have been installed 10 years ago along 

In [49]:
column_names = ['Borough', 'Latitude', 'Longitude', 'Independent Schools', 'Crime Per Pop'] 

# instantiate the dataframe
boroughs = pd.DataFrame(columns=column_names)
boroughs

,Borough,Latitude,Longitude,Independent Schools,Crime Per Pop


In [50]:
london_data[0]['area']

'Barking and Dagenham'

In [51]:
for data in london_data:
    name = data['area']    
    lat = data['latitude']
    long = data['longitude']
    independent = data['independent-schools']
    crime = data['crime-per-pop']
    
    boroughs = boroughs.append({'Borough': name,
                                          'Latitude': lat,
                                          'Longitude': long,
                                          'Independent Schools': independent,
                                           'Crime Per Pop': crime}, ignore_index=True)

In [52]:
boroughs

,Borough,Latitude,Longitude,Independent Schools,Crime Per Pop
0,Barking and Dagenham,51.545277,0.133528,4,70.0383
1,Barnet,51.560700,-0.210017,36,55.3106
2,Bexley,51.625200,0.140355,5,45.6938
3,Brent,51.454900,-0.267821,14,69.9699
4,Bromley,51.371999,0.051524,19,52.5862
5,Camden,51.546394,-0.157424,33,104.701
6,City of London,51.529000,-0.092171,4,No data
7,Croydon,51.371400,-0.087157,24,66.6934
8,Ealing,51.513000,-0.331026,21,67.3606
9,Enfield,51.653800,-0.087272,12,58.2613


As per the lab, I'll get the geolocation of London itelf

In [53]:
address = 'London, United Kingdom'
geolocator = Nominatim(user_agent="MyGeocoder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [56]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, name in zip(boroughs['Latitude'], boroughs['Longitude'], boroughs['Borough']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### Foursquare

In [57]:
CLIENT_ID = '4KFYAOFDVDOA1JSXNBWKTGLMAQZJAJBJ1G32CPQYW4VN5Z2P' # your Foursquare ID
CLIENT_SECRET = '1MUUCZTL1MRTUORURDZFOLPD0N3VTNYULPWZ15WABVO4LR5Q' # your Foursquare Secret
VERSION = '20200404' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 4KFYAOFDVDOA1JSXNBWKTGLMAQZJAJBJ1G32CPQYW4VN5Z2P
CLIENT_SECRET:1MUUCZTL1MRTUORURDZFOLPD0N3VTNYULPWZ15WABVO4LR5Q


In [58]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
london_venues = getNearbyVenues(names=boroughs['Borough'],
                                   latitudes=boroughs['Latitude'],
                                   longitudes=boroughs['Longitude']
                                  )


Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
City of London
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [61]:
print(london_venues.shape)
london_venues.head()

(1867, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.545277,0.133528,Co-op Food,51.540093,0.127522,Grocery Store
1,Barking and Dagenham,51.545277,0.133528,Becontree London Underground Station,51.540335,0.127061,Metro Station
2,Barking and Dagenham,51.545277,0.133528,Londis,51.547309,0.145934,Convenience Store
3,Barking and Dagenham,51.545277,0.133528,Children And Baby Gifts,51.538942,0.123285,Gift Shop
4,Barnet,51.560700,-0.210017,Virgin Active,51.559503,-0.210219,Gym / Fitness Center


In [62]:
london_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,4,4,4,4,4,4
Barnet,47,47,47,47,47,47
Bexley,2,2,2,2,2,2
Brent,11,11,11,11,11,11
Bromley,12,12,12,12,12,12
Camden,94,94,94,94,94,94
City of London,100,100,100,100,100,100
Croydon,63,63,63,63,63,63
Ealing,39,39,39,39,39,39


In [63]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 250 uniques categories.


#### Some Data Cleaning...

In [64]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
london_onehot['Borough'] = london_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Canal,Canal Lock,Caribbean Restaurant,Castle,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Concert Hall,Construction & Landscaping,Convenience Store,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [33]:
london_onehot.shape

(1867, 251)

In [35]:
london_grouped = london_onehot.groupby('Borough').mean().reset_index()
london_grouped.head()

,Borough,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Canal,Canal Lock,Caribbean Restaurant,Castle,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Concert Hall,Construction & Landscaping,Convenience Store,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,South Indian Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yakitori Restaurant,Yoga Studio
0,Barking and Dagenham,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0

In [37]:
london_grouped.shape

(33, 251)

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Barking and Dagenham,Gift Shop,Grocery Store,Convenience Store,Metro Station,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Yoga Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Farm,Falafel Restaurant,Gaming Cafe,Exhibit
1,Barnet,Grocery Store,Hotel,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Bus Stop,Park,Italian Restaurant,Gym / Fitness Center,Café,Sushi Restaurant,Pub,Sandwich Place,Clothing Store,Ethiopian Restaurant,Sporting Goods Shop,Convenience Store,Supermarket,Afghan Restaurant,Health & Beauty Service
2,Bexley,Steakhouse,Athletics & Sports,Farm,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Garden Center,Exhibit,Ethiopian Restaurant
3,Brent,Park,Café,Pub,Soccer Field,Tennis Court,Golf Course,Athletics & Sports,Yoga Studio,Fish Market,Fish & Chips Shop,Flea Market,Farmers Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fast Food Restaurant,Farm,Furniture / Home Store
4,Bromley,Pub,Gas Station,Supermarket,Fast Food Restaurant,Bus Station,Bus Stop,Molecular Gastronomy Restaurant,Coffee Shop,Food,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio,Food & Drink Shop,Farmers Market,Food Truck,French Restaurant,Food Court,Farm,Furniture / Home Store


### Clustering Time

In [112]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 3, 1, 0, 1, 1, 1, 1, 1])

In [113]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = boroughs

# merge toronto_grouped with toronto_data to add latitude/longitude for each Borough
london_merged = london_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

london_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Barking and Dagenham,51.545277,0.133528,4,70.0383,2,Gift Shop,Grocery Store,Convenience Store,Metro Station,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Yoga Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Farm,Falafel Restaurant,Gaming Cafe,Exhibit
1,Barnet,51.560700,-0.210017,36,55.3106,4,Grocery Store,Hotel,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Bus Stop,Park,Italian Restaurant,Gym / Fitness Center,Café,Sushi Restaurant,Pub,Sandwich Place,Clothing Store,Ethiopian Restaurant,Sporting Goods Shop,Convenience Store,Supermarket,Afghan Restaurant,Health & Beauty Service
2,Bexley,51.625200,0.140355,5,45.6938,3,Steakhouse,Athletics & Sports,Farm,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Garden Center,Exhibit,Ethiopian Restaurant
3,Brent,51.454900,-0.267821,14,69.9699,1,Park,Café,Pub,Soccer Field,Tennis Court,Golf Course,Athletics & Sports,Yoga Studio,Fish Market,Fish & Chips Shop,Flea Market,Farmers Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fast Food Restaurant,Farm,Furniture / Home Store
4,Bromley,51.371999,0.051524,19,52.5862,0,Pub,Gas Station,Supermarket,Fast Food Restaurant,Bus Station,Bus Stop,Molecular Gastronomy Restaurant,Coffee Shop,Food,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio,Food & Drink Shop,Farmers Market,Food Truck,French Restaurant,Food Court,Farm,Furniture / Home Store


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Borough'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [115]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + [3] + [4] + list(range(5, london_merged.shape[1]))]]

,Borough,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,Bromley,19,52.5862,0,Pub,Gas Station,Supermarket,Fast Food Restaurant,Bus Station,Bus Stop,Molecular Gastronomy Restaurant,Coffee Shop,Food,Fish & Chips Shop,Fish Market,Flea Market,Yoga Studio,Food & Drink Shop,Farmers Market,Food Truck,French Restaurant,Food Court,Farm,Furniture / Home Store


### Cluster 2

In [116]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + [3] + [4] +  list(range(5, london_merged.shape[1]))]]

,Borough,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
3,Brent,14,69.9699,1,Park,Café,Pub,Soccer Field,Tennis Court,Golf Course,Athletics & Sports,Yoga Studio,Fish Market,Fish & Chips Shop,Flea Market,Farmers Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fast Food Restaurant,Farm,Furniture / Home Store
5,Camden,33,104.701,1,Pub,Coffee Shop,Café,Market,Bar,Hotel,Grocery Store,Bus Stop,Burger Joint,Park,Bakery,Convenience Store,Fast Food Restaurant,Gastropub,Vegetarian / Vegan Restaurant,Deli / Bodega,American Restaurant,Cupcake Shop,Salon / Barbershop,Pharmacy
6,City of London,4,No data,1,Coffee Shop,Pub,Gym / Fitness Center,Food Truck,Vietnamese Restaurant,Cocktail Bar,Café,Italian Restaurant,Yoga Studio,Bar,Hotel,Japanese Restaurant,Restaurant,Burger Joint,Breakfast Spot,Bike Shop,Ramen Restaurant,Asian Restaurant,Costume Shop,Indie Movie Theater
7,Croydon,24,66.6934,1,Coffee Shop,Pub,Platform,Mediterranean Restaurant,Café,Hotel,Portuguese Restaurant,Indian Restaurant,Grocery Store,Tram Station,Italian Restaurant,Sports Club,Pizza Place,Burger Joint,Burrito Place,Bus Station,Bus Stop,Spanish Restaurant,Food Court,Caribbean Restaurant
8,Ealing,21,67.3606,1,Pub,Café,Supermarket,Grocery Store,Coffee Shop,Convenience Store,Fast Food Restaurant,Train Station,Hotel,Liquor Store,Park,Indian Restaurant,Golf Course,Fish & Chips Shop,Diner,Gym,Pharmacy,Bakery,Bagel Shop,Furniture / Home Store
9,Enfield,12,58.2613,1,Coffee Shop,Pub,Clothing Store,Café,Optical Shop,Supermarket,Train Station,Fish & Chips Shop,Shopping Mall,Mobile Phone Shop,Bookstore,Gift Shop,Indian Restaurant,Sandwich Place,Grocery Store,Video Game Store,Pharmacy,Department Store,Turkish Restaurant,Electronics Store
10,Greenwich,14,71.4852,1,Coffee Shop,Fast Food Restaurant,Supermarket,Pub,Clothing Store,Pharmacy,Hotel,Plaza,Sandwich Place,Breakfast Spot,Boat or Ferry,Steakhouse,Bookstore,Discount Store,Pizza Place,Bakery,Burger Joint,Furniture / Home Store,Bus Stop,Café
11,Hackney,33,90.8968,1,Pub,Café,Bakery,Cocktail Bar,Restaurant,Brewery,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Sandwich Place,Butcher,Market,Bookstore,Italian Restaurant,Theater,Roof Deck,Gym / Fitness Center,Park,Grocery Store
12,Hammersmith and Fulham,22,95.1376,1,Pub,Coffee Shop,Sandwich Place,Gym / Fitness Center,Bakery,Café,Thai Restaurant,Italian Restaurant,Gastropub,Hotel,Park,Cocktail Bar,Tapas Restaurant,Portuguese Restaurant,Farmers Market,Pizza Place,Indian Restaurant,Turkish Restaurant,Bar,Fish Market
13,Haringey,13,85.9496,1,Pub,Clothing Store,Fast Food Restaurant,Coffee Shop,Café,Turkish Restaurant,Park,Mediterranean Restaurant,Greek Restaurant,Sandwich Place,Pharmacy,Indian Restaurant,Chinese Restaurant,Supermarket,Gastropub,Bookstore,Stadium,Garden Center,Shopping Mall,Market


### Cluster 3

In [117]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + [3] + [4] +  list(range(5, london_merged.shape[1]))]]

,Borough,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Barking and Dagenham,4,70.0383,2,Gift Shop,Grocery Store,Convenience Store,Metro Station,Flea Market,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Yoga Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Farm,Falafel Restaurant,Gaming Cafe,Exhibit


### Cluster 4

In [118]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[0] + [3] + [4] +  list(range(5, london_merged.shape[1]))]]

,Borough,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
2,Bexley,5,45.6938,3,Steakhouse,Athletics & Sports,Farm,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Food & Drink Shop,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Garden Center,Exhibit,Ethiopian Restaurant


### Cluster 5

In [119]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[0] + [3] + [4] +  list(range(5, london_merged.shape[1]))]]

,Borough,Independent Schools,Crime Per Pop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Barnet,36,55.3106,4,Grocery Store,Hotel,Fast Food Restaurant,Coffee Shop,Breakfast Spot,Bus Stop,Park,Italian Restaurant,Gym / Fitness Center,Café,Sushi Restaurant,Pub,Sandwich Place,Clothing Store,Ethiopian Restaurant,Sporting Goods Shop,Convenience Store,Supermarket,Afghan Restaurant,Health & Beauty Service
15,Havering,7,57.7271,4,Bar,Pub,Gym / Fitness Center,Fast Food Restaurant,Grocery Store,Auto Garage,Park,Food,Food & Drink Shop,Farm,Flea Market,Fish Market,Fish & Chips Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Farmers Market,Yoga Studio,Gaming Cafe
16,Hillingdon,9,63.7708,4,Chinese Restaurant,Fast Food Restaurant,Pub,Grocery Store,Coffee Shop,Plaza,Park,Bus Stop,Food,Flea Market,Farm,Food & Drink Shop,Food Court,Fish Market,Fish & Chips Shop,Farmers Market,Food Truck,Yoga Studio,Falafel Restaurant,Fried Chicken Joint
17,Hounslow,13,70.5288,4,Coffee Shop,Grocery Store,Clothing Store,Fast Food Restaurant,Indian Restaurant,Hotel,Bus Stop,Pub,Supermarket,Sandwich Place,Chinese Restaurant,Pharmacy,Discount Store,Bakery,Video Game Store,Gift Shop,Outlet Store,Sporting Goods Shop,Sports Bar,Metro Station
22,Lewisham,8,68.0751,4,Grocery Store,Supermarket,Park,Pub,Café,Furniture / Home Store,Platform,Coffee Shop,Italian Restaurant,Portuguese Restaurant,Sandwich Place,Shopping Mall,Cocktail Bar,Shopping Plaza,Greek Restaurant,Bakery,Trail,Pharmacy,Japanese Restaurant,Bus Stop
23,Merton,17,52.7295,4,Café,Supermarket,Fast Food Restaurant,Grocery Store,Italian Restaurant,Pizza Place,Garden Center,Park,Pet Store,Bar,Bakery,Sandwich Place,Burger Joint,Coffee Shop,Indian Restaurant,Pool,Dive Bar,Food Truck,Food Court,Food & Drink Shop
24,Newham,9,76.7162,4,Hotel,Pub,Coffee Shop,Sandwich Place,Chinese Restaurant,Restaurant,Hotel Bar,Theater,Wine Bar,American Restaurant,Bridge,Gas Station,Farm,Grocery Store,Tapas Restaurant,Pharmacy,Airport Service,Bed & Breakfast,Park,Bakery
25,Redbridge,19,55.7627,4,Fast Food Restaurant,Clothing Store,Grocery Store,Supermarket,Coffee Shop,Gym / Fitness Center,Indian Restaurant,Bakery,Sandwich Place,Hotel,Pub,Department Store,Shoe Store,Stationery Store,Pizza Place,Bookstore,Portuguese Restaurant,Burger Joint,Furniture / Home Store,Fried Chicken Joint
